In [51]:
import pandas as pd 
import lightgbm as lgb
from lightgbm import LGBMRegressor
import optuna
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split , cross_val_score
from sklearn.metrics import mean_squared_error
import warnings
pd.set_option('display.max_columns', None)
import re 

In [52]:
train = pd.read_csv('/kaggle/input/playground-series-s4e9/train.csv')
test = pd.read_csv('/kaggle/input/playground-series-s4e9/test.csv')
sample_submission = pd.read_csv('/kaggle/input/playground-series-s4e9/sample_submission.csv')

In [53]:
train.head(2)

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
0,0,MINI,Cooper S Base,2007,213000,Gasoline,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,A/T,Yellow,Gray,None reported,Yes,4200
1,1,Lincoln,LS V8,2002,143250,Gasoline,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,A/T,Silver,Beige,At least 1 accident or damage reported,Yes,4999


In [54]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188533 entries, 0 to 188532
Data columns (total 13 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   id            188533 non-null  int64 
 1   brand         188533 non-null  object
 2   model         188533 non-null  object
 3   model_year    188533 non-null  int64 
 4   milage        188533 non-null  int64 
 5   fuel_type     183450 non-null  object
 6   engine        188533 non-null  object
 7   transmission  188533 non-null  object
 8   ext_col       188533 non-null  object
 9   int_col       188533 non-null  object
 10  accident      186081 non-null  object
 11  clean_title   167114 non-null  object
 12  price         188533 non-null  int64 
dtypes: int64(4), object(9)
memory usage: 18.7+ MB


In [55]:
train['fuel_type'].isnull().sum()

5083

In [56]:
train[train['fuel_type'].isnull()].sample(5)

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
173219,173219,Tesla,Model X 75D,2017,5800,NaN,518.0HP Electric Motor Electric Fuel System,A/T,White,Black,None reported,Yes,31950
67907,67907,Tesla,Model 3 Standard Range Plus,2019,31500,NaN,Electric Motor Electric Fuel System,A/T,White,Blue,None reported,Yes,54000
65214,65214,BMW,i3 Base,2019,26000,NaN,170.0HP 0.65L Electric Motor Electric Fuel System,1-Speed A/T,Red,Beige,None reported,Yes,39000
142613,142613,Tesla,Model Y Performance,2016,69700,NaN,455.0HP Electric Motor Electric Fuel System,1-Speed A/T,Gray,Black,None reported,Yes,32000
58191,58191,Tesla,Model S 100D,2016,102260,NaN,Electric Motor Electric Fuel System,1-Speed A/T,Silver,Black,None reported,Yes,64000


In [57]:
train[(train['brand'].isin(['Tesla','Audi','Porsche']) == False) & (train['fuel_type'].isnull())]

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
176,176,Nissan,Leaf S,2020,14330,NaN,Electric,1-Speed Automatic,Brilliant Silver Metallic,Black,None reported,NaN,52999
418,418,Chevrolet,Bolt EV LT,2016,89000,NaN,200.0HP Electric Motor Electric Fuel System,A/T,Red,Black,None reported,Yes,14500
714,714,Nissan,Leaf SV PLUS,2023,7089,NaN,214.0HP Electric Motor Electric Fuel System,1-Speed A/T,Black,Blue,None reported,Yes,9500
797,797,Ford,Mustang Mach-E GT,2022,7661,NaN,Electric,M/T,Star White,Black Onyx,None reported,Yes,68969
902,902,Rivian,R1S Adventure Package,2023,8400,NaN,835.0HP Electric Motor Electric Fuel System,1-Speed A/T,Gray,White,None reported,Yes,93000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
188362,188362,Volkswagen,e-Golf SE,2021,22185,NaN,Electric,1-Speed Automatic,Brilliant Silver Metallic,Graphite w/Gun Metal,None reported,NaN,34999
188368,188368,Ford,Mustang Mach-E Premium,2022,2058,NaN,Standard Range Battery,Automatic CVT,Red,Black,None reported,NaN,27785
188455,188455,Ford,Mustang Mach-E GT,2022,8000,NaN,480.0HP Electric Motor Electric Fuel System,A/T,Orange,White,None reported,Yes,115000
188462,188462,Ford,Mustang Mach-E GT,2023,9000,NaN,480.0HP Electric Motor Electric Fuel System,A/T,Orange,Black,None reported,Yes,85000


In [58]:
train['fuel_type'].unique()

array(['Gasoline', 'E85 Flex Fuel', nan, 'Hybrid', 'Diesel',
       'Plug-In Hybrid', '–', 'not supported'], dtype=object)

In [59]:
train[(train['fuel_type'].isin(['–']) == True)]

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
532,532,Dodge,Challenger R/T Scat Pack,2021,35000,–,–,8-Speed A/T,Blue,Gray,None reported,Yes,48000
1155,1155,Ford,Mustang GT Premium,2017,143600,–,–,6-Speed M/T,White,Black,At least 1 accident or damage reported,Yes,31000
1307,1307,Toyota,Land Cruiser Base,1994,138033,–,–,A/T,Green,Beige,None reported,Yes,11999
1362,1362,Ford,Mustang EcoBoost Premium,2016,43000,–,–,A/T,Blue,Black,None reported,Yes,15500
1741,1741,Chevrolet,Protege DX,1993,94200,–,–,A/T,Red,Gray,None reported,Yes,7599
...,...,...,...,...,...,...,...,...,...,...,...,...,...
187399,187399,Mazda,MX-5 Miata Base,1993,90200,–,–,5-Speed M/T,Gray,Black,At least 1 accident or damage reported,Yes,11500
187495,187495,Nissan,Pathfinder SL,2014,87305,–,284.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,A/T,Brown,Gray,At least 1 accident or damage reported,Yes,10000
187664,187664,Mercedes-Benz,SLS AMG Base,1993,14586,–,–,A/T,Red,Silver,None reported,Yes,65000
187820,187820,Chrysler,Pacifica Touring,2017,57300,–,–,9-Speed A/T,Blue,Black,None reported,Yes,31700


In [60]:
train[(train['fuel_type'].isin(['not supported']) == True)]

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
4823,4823,Chevrolet,Suburban LT,2011,109646,not supported,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,6-Speed A/T,Black,Beige,None reported,Yes,18998
18000,18000,Porsche,Camaro 2SS,2015,67000,not supported,240.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,Black,None reported,Yes,11500
19182,19182,Nissan,Mirai Limited,2000,131000,not supported,182.0HP Electric Motor Hydrogen Fuel,A/T,Silver,Black,None reported,Yes,5000
27199,27199,Subaru,Outback 2.5i Limited,2017,116000,not supported,175.0HP 2.5L 4 Cylinder Engine Gasoline Fuel,CVT Transmission,Gray,Black,At least 1 accident or damage reported,Yes,27499
33030,33030,Nissan,Mirai Limited,2009,137333,not supported,182.0HP Electric Motor Hydrogen Fuel,A/T,White,Gray,At least 1 accident or damage reported,Yes,12999
60717,60717,Chevrolet,Silverado 1500 RST,2020,19200,not supported,355.0HP 5.3L 8 Cylinder Engine Gasoline Fuel,10-Speed A/T,White,Black,At least 1 accident or damage reported,Yes,58000
74096,74096,Toyota,Mirai Limited,2021,79876,not supported,182.0HP Electric Motor Hydrogen Fuel,A/T,Red,Black,None reported,Yes,15000
86897,86897,Lexus,IS 350 F Sport,2022,9500,not supported,311.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,A/T,Gray,Black,None reported,Yes,60000
94425,94425,Porsche,Cayenne Turbo GT,2018,52000,not supported,550.0HP 4.8L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Gray,Gray,At least 1 accident or damage reported,Yes,35000
127066,127066,Toyota,Mirai Limited,2016,58664,not supported,151.0HP Electric Motor Hydrogen Fuel,CVT Transmission,Black,Black,At least 1 accident or damage reported,Yes,22500


In [61]:
train['fuel_type'] = train['fuel_type'].replace('not supported', 'Electric')
train['fuel_type'] = train['fuel_type'].replace('–', 'Gasoline')
train['fuel_type'] = train['fuel_type'].fillna('Electric')

In [62]:
test['fuel_type'] = test['fuel_type'].replace('not supported', 'Electric')
test['fuel_type'] = test['fuel_type'].replace('–', 'Gasoline')
test['fuel_type'] = test['fuel_type'].fillna('Electric')

In [63]:
train['fuel_type'].unique()


array(['Gasoline', 'E85 Flex Fuel', 'Electric', 'Hybrid', 'Diesel',
       'Plug-In Hybrid'], dtype=object)

In [64]:
test['fuel_type'].unique()

array(['Gasoline', 'Hybrid', 'Diesel', 'E85 Flex Fuel', 'Electric',
       'Plug-In Hybrid'], dtype=object)

In [65]:
train.columns

Index(['id', 'brand', 'model', 'model_year', 'milage', 'fuel_type', 'engine',
       'transmission', 'ext_col', 'int_col', 'accident', 'clean_title',
       'price'],
      dtype='object')

In [66]:
a = train['accident'].isna().sum()

In [67]:
( a / len(train['accident']) ) *100

1.3005680703112983

In [68]:
train['accident'].unique()

array(['None reported', 'At least 1 accident or damage reported', nan],
      dtype=object)

In [69]:
train['accident'] = train['accident'].fillna('None reported')

In [70]:
test['accident'] = test['accident'].fillna('None reported')

In [71]:
train[train['accident'].isna()]

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price


In [72]:
train['accident'] = train['accident'].fillna('No')

In [73]:
test['accident'] = train['accident'].fillna('No')

In [74]:
train[train['clean_title'].isna()].head(3)

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
20,20,Land,Rover Defender S,2023,7978,Gasoline,5.2L V10 40V PDI DOHC,8-Speed Automatic,Santorini Black Metallic,Black,None reported,NaN,41998
45,45,RAM,1500 Laramie,2022,27352,Gasoline,5.7L V8 16V MPFI OHV,8-Speed Automatic,Diamond Black,Black,None reported,NaN,42599
51,51,Tesla,Model 3 Long Range,2020,38360,Electric,Dual Motor - Standard,Automatic,White,Black,None reported,NaN,59598


In [75]:
train['clean_title'] = train['clean_title'].fillna('No')

In [76]:
test['clean_title'] = test['clean_title'].fillna('No')

In [77]:
train['clean_title'].isna()

0         False
1         False
2         False
3         False
4         False
          ...  
188528    False
188529    False
188530    False
188531    False
188532    False
Name: clean_title, Length: 188533, dtype: bool

In [78]:
train['clean_title'].unique()

array(['Yes', 'No'], dtype=object)

In [79]:
train.isna().sum()

id              0
brand           0
model           0
model_year      0
milage          0
fuel_type       0
engine          0
transmission    0
ext_col         0
int_col         0
accident        0
clean_title     0
price           0
dtype: int64

In [80]:
test.isna().sum()

id              0
brand           0
model           0
model_year      0
milage          0
fuel_type       0
engine          0
transmission    0
ext_col         0
int_col         0
accident        0
clean_title     0
dtype: int64

In [81]:
# Define a function to extract engine attributes
def extract_engine_attributes(engine_str):
    horsepower = re.search(r'(\d+\.\d+)HP|\d+\.\d+', engine_str)
    displacement = re.search(r'(\d+\.\d+)L|(\d+\.\d+) Liter', engine_str)
    return horsepower.group(1) if horsepower else '',\
           displacement.group(1) if displacement else ''

# Apply the function to create new columns
train[['horsepower', 'engine_displacement']] = train['engine'].apply(extract_engine_attributes).apply(pd.Series)
test[['horsepower', 'engine_displacement']] = test['engine'].apply(extract_engine_attributes).apply(pd.Series)

In [82]:
train.sample(3)

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price,horsepower,engine_displacement
104261,104261,Ford,F-150 XLT,2018,98000,Gasoline,325.0HP 2.7L V6 Cylinder Engine Gasoline Fuel,A/T,Gray,Gray,None reported,Yes,19000,325.0,2.7
166294,166294,BMW,M4 Base,2020,28000,Gasoline,425.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,6-Speed M/T,White,Black,None reported,No,41999,425.0,3.0
13908,13908,Mazda,MX-5 Miata Base,2001,153000,Gasoline,173.0HP 2.7L V6 Cylinder Engine Gasoline Fuel,A/T,Blue,Beige,None reported,Yes,17000,173.0,2.7


In [84]:
categorical_columns=[feature for feature in train.columns if train[feature].dtype=='O']
numerical_columns=[feature for feature in train.columns if train[feature].dtype!='O']

In [85]:
label=LabelEncoder()
categorical_columns = train.select_dtypes(include ='object').columns
for column in categorical_columns:
    train[column] = train[column].astype(str)
    test[column] = test[column].astype(str)
    test[column] = label.fit_transform(test[column])
    train[column] = label.fit_transform(train[column])

In [86]:
train.head()

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price,horsepower,engine_displacement
0,0,31,495,2007,213000,3,116,38,312,71,1,1,4200,51,7
1,1,28,930,2002,143250,3,366,38,263,10,0,1,4999,118,27
2,2,9,1575,2002,136731,1,640,38,38,71,1,1,13900,174,38
3,3,16,758,2017,19500,3,863,49,29,14,1,1,45000,238,36
4,4,36,1077,2021,7388,3,259,23,29,10,1,1,97500,84,10


In [88]:
X = train.drop(columns=['price'], axis =1)
y = train['price']
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state = 1
)

In [89]:
'''def objective_lgbm(trial):

    lgbm_params = {
    'n_estimators': trial.suggest_int('n_estimators', 1000, 2000),
    'num_leaves': trial.suggest_int('num_leaves', 115, 250),
    'max_depth': trial.suggest_int('max_depth', 25, 40),
    'cat_smooth': trial.suggest_int('cat_smooth', 50, 200),
    'learning_rate': trial.suggest_float('learning_rate', 1e-4, 0.01, log = True),
    'subsample': trial.suggest_float('subsample', 0.7, 0.9,log = True),
    'colsample_bytree': trial.suggest_float('colsample_bytree', 0.4, 0.9, log = True),
    'min_split_gain': trial.suggest_float('min_split_gain', 1e-4, 0.8, log = True),
    'min_child_weight': trial.suggest_int('min_child_weight', 30, 90),
    'lambda_l2': trial.suggest_float('lambda_l2', 1e-8, 1e-5,log = True),
    'lambda_l1': trial.suggest_float('lambda_l1', 1e-8, 1e-5,log = True),
    #'scale_pos_weight': trial.suggest_uniform('scale_pos_weight', 1.0, 12.0),
    'max_bin': trial.suggest_int('max_bin', 500, 900),
    }
    

    # Fit the model
    model_lgbm = LGBMRegressor(
        **lgbm_params, random_state= 1, loss_function="RMSE", verbose=-1
    )

    model_lgbm.fit(X_train, y_train)

    # Predict and calculate mean squared error score
    y_pred = model_lgbm.predict(X_test)

    return mean_squared_error(y_test, y_pred, squared = False)'''

'def objective_lgbm(trial):\n\n    lgbm_params = {\n    \'n_estimators\': trial.suggest_int(\'n_estimators\', 1000, 2000),\n    \'num_leaves\': trial.suggest_int(\'num_leaves\', 115, 250),\n    \'max_depth\': trial.suggest_int(\'max_depth\', 25, 40),\n    \'cat_smooth\': trial.suggest_int(\'cat_smooth\', 50, 200),\n    \'learning_rate\': trial.suggest_float(\'learning_rate\', 1e-4, 0.01, log = True),\n    \'subsample\': trial.suggest_float(\'subsample\', 0.7, 0.9,log = True),\n    \'colsample_bytree\': trial.suggest_float(\'colsample_bytree\', 0.4, 0.9, log = True),\n    \'min_split_gain\': trial.suggest_float(\'min_split_gain\', 1e-4, 0.8, log = True),\n    \'min_child_weight\': trial.suggest_int(\'min_child_weight\', 30, 90),\n    \'lambda_l2\': trial.suggest_float(\'lambda_l2\', 1e-8, 1e-5,log = True),\n    \'lambda_l1\': trial.suggest_float(\'lambda_l1\', 1e-8, 1e-5,log = True),\n    #\'scale_pos_weight\': trial.suggest_uniform(\'scale_pos_weight\', 1.0, 12.0),\n    \'max_bin\': tr

In [91]:
'''study = optuna.create_study(direction='minimize')
study.optimize(objective_lgbm, n_trials=50)'''

"study = optuna.create_study(direction='minimize')\nstudy.optimize(objective_lgbm, n_trials=50)"

In [92]:
'''best_params = study.best_params
print("Best parameters:", best_params)'''

'best_params = study.best_params\nprint("Best parameters:", best_params)'

In [93]:
lgbm_params = {
    'n_estimators': 1247,
    'num_leaves': 123,
    'max_depth': 28,
    'cat_smooth': 143,
    'learning_rate': 0.0027129249008214056,
    'subsample': 0.7977379135998542,
    'colsample_bytree': 0.4588317424928748,
    'min_split_gain': 0.003110663752733955,
    'min_child_weight': 85,
    'lambda_l2': 3.3613296039345123e-08,
    'lambda_l1': 8.445494309658136e-08,
    'max_bin' :777
}

In [94]:
model = LGBMRegressor(**lgbm_params)
model.fit(X_train, y_train)

[LightGBM] [Warning] lambda_l2 is set=3.3613296039345123e-08, reg_lambda=0.0 will be ignored. Current value: lambda_l2=3.3613296039345123e-08
[LightGBM] [Warning] lambda_l1 is set=8.445494309658136e-08, reg_alpha=0.0 will be ignored. Current value: lambda_l1=8.445494309658136e-08
[LightGBM] [Warning] lambda_l2 is set=3.3613296039345123e-08, reg_lambda=0.0 will be ignored. Current value: lambda_l2=3.3613296039345123e-08
[LightGBM] [Warning] lambda_l1 is set=8.445494309658136e-08, reg_alpha=0.0 will be ignored. Current value: lambda_l1=8.445494309658136e-08
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.047889 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4015
[LightGBM] [Info] Number of data points in the train set: 141399, number of used features: 14
[LightGBM] [Info] Start training from score 44092.505173


LGBMRegressor(cat_smooth=143, colsample_bytree=0.4588317424928748,
              lambda_l1=8.445494309658136e-08, lambda_l2=3.3613296039345123e-08,
              learning_rate=0.0027129249008214056, max_bin=777, max_depth=28,
              min_child_weight=85, min_split_gain=0.003110663752733955,
              n_estimators=1247, num_leaves=123, subsample=0.7977379135998542)

In [95]:
y_pred = model.predict(test)

[LightGBM] [Warning] lambda_l2 is set=3.3613296039345123e-08, reg_lambda=0.0 will be ignored. Current value: lambda_l2=3.3613296039345123e-08
[LightGBM] [Warning] lambda_l1 is set=8.445494309658136e-08, reg_alpha=0.0 will be ignored. Current value: lambda_l1=8.445494309658136e-08


In [96]:
y_pred

array([18747.0934658 , 71539.75478745, 62715.34410591, ...,
       25502.89830595, 21501.42572245, 36794.60214614])

In [97]:
ids =  sample_submission['id']

In [98]:
sub = pd.DataFrame({'id' : ids, 
                   'price' : y_pred})

In [99]:
sub

,id,price
0,188533,18747.093466
1,188534,71539.754787
2,188535,62715.344106
3,188536,34566.496347
4,188537,31594.635824
...,...,...
125685,314218,32856.020886
125686,314219,55145.377584
125687,314220,25502.898306
125688,314221,21501.425722


In [100]:
sub.to_csv('submission.csv', index = False)